# Exploratory analysis

We aren't focusing on defining metrics or data cleaning in this workshop, so our first step will be exploratory analysis on a prepared dataset.  This will involve looking at the data and then examining each variable to see if we can identify any interesting patterns.

We will use a range of Python libraries to help us explore, visualise and interact with the data. We could install these into the running environment, one by one. For example: 

`!pip install numpy`

would install `numpy`. However, it is good practice to keep track of all your requirements and specific versions in one location. The explicit versions of the libraries we will use are listed in the `requirements.txt` file, which you can open from the file viewer on the left hand side of the screen. We install all these requirements into our environment below:

In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 15.7 MB 5.0 MB/s                  | 5.6 MB 5.0 MB/s eta 0:00:03
     |████████████████████████████████| 28.4 MB 16.2 MB/s            ��███▋   | 25.4 MB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 48.1 MB/s            �█████████▉                 | 5.3 MB 48.1 MB/s eta 0:00:01��██████████████████     | 9.7 MB 48.1 MB/s eta 0:00:01
     |████████████████████████████████| 727 kB 54.1 MB/s                 | 256 kB 54.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 49.9 MB/s            
     |████████████████████████████████| 63.8 MB 3.1 MB/s                         | 3.0 MB 12.9 MB/s eta 0:00:050511.0 MB 12.9 MB/s eta 0:00:05 20.1 MB 12.9 MB/s eta 0:00:04��████▏                  | 26.3 MB 12.9 MB/s eta 0:00:03         | 28.0 MB 12.9 MB/s eta 0:00:03��███                | 31.9 MB 12.9 MB/s eta 0:00:03             | 37.4 MB 12.9 MB/s eta 0:00:03           | 41.4 MB 12.9 MB/s eta 0:00:02    |███████████████████

We can then import the installed libraries into our notebook environment like so:

In [2]:
import pandas as pd
import numpy as np

Before we get started with exploring our data, we first need to load the data. The data is stored in an S3 bucket, and we connect to it using the `boto3` library. The `boto3` library was built into the `Standard Data Science` notebook image which we selected from the spawner page. As such, it is already installed in our environment. (However, it's also good practice to keep track of every library you use in your `requirements.txt` file - go ahead and add `boto3` to the `requirements.txt`).

In [3]:
import boto3

boto3 expects all requests to be signed, meaning you have to authenticate and provide credentials (keys, ids) to connect to data. However we are connecting to public data which is in a open s3 bucket, so we use the `UNSIGNED` signature from botocore to allow us to access the data without providing credentials. 

In [4]:
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.client('s3', config = Config(signature_version=UNSIGNED))
s3.download_file('fraud-detection-dataset', 'fraud-cleaned-sample.parquet', 'fraud-cleaned-sample.parquet')

We'll start by reading in our fraud dataset and looking at the column names:

In [5]:
df = pd.read_parquet("fraud-cleaned-sample.parquet")
df.columns

Index(['timestamp', 'label', 'user_id', 'amount', 'merchant_id', 'trans_type',
       'foreign', 'interarrival'],
      dtype='object')

When dealing with an unfamiliar dataset, we first need to understand what data we've collected.  If we're lucky, our organization will publish a data catalog or data glossary that explains what each dataset contains.  If we don't have this extra metadata, we'll need to explore the dataset manually and document our observations.  Sometimes the column names will be self-explanatory, but if they aren't, we'll have to dig deeper to make sense of them.  A good second step is examining the types of each column.

In [6]:
df.dtypes

timestamp         int64
label            string
user_id           int32
amount          float32
merchant_id       int32
trans_type       string
foreign            bool
interarrival    float64
dtype: object

Sometimes, `string`-valued attributes in structured data are free-form text, but some string-valued variables are [categorical variables](https://en.wikipedia.org/wiki/Categorical_variable).  We can usually identify these by looking at unique values to identify attributes with a small number of possible values.

In [7]:
list(df["label"].unique())

['legitimate', 'fraud']

In [8]:
list(df["trans_type"].unique())

['chip_and_pin', 'contactless', 'online', 'swipe', 'manual']

If we're dealing with messy data, categorical variables may have a large number of unique values, some of which are spurious and may occur only once.  In this case, we don't want to examine every unique value but we can look at the top few values to get a sense for what the range of values _should_ be.  In this next cell, we're looking at the three most common transaction types.

In [9]:
df[["timestamp", "trans_type"]].groupby(["trans_type"]).count().nlargest(3, "timestamp")

,timestamp
trans_type,
online,882304
contactless,615511
chip_and_pin,493596


We can look at the most common transaction types for each class of transaction, as well.

In [10]:
df[df["label"] == "fraud"][["timestamp", "trans_type"]].groupby(
    ["trans_type"]
).count(
).nlargest(3, "timestamp")

,timestamp
trans_type,
online,23335
manual,16124
contactless,2346


In [11]:
df[df["label"] == "legitimate"][["timestamp", "trans_type"]].groupby(
    ["trans_type"]
).count(
).nlargest(3, "timestamp")

,timestamp
trans_type,
online,858969
contactless,613165
chip_and_pin,491303


# Transaction type distribution

Let's look at the transaction type first to see what the distribution looks like for legitimate and fraud transactions.

In [12]:
pt = pd.pivot_table(df[["label", "trans_type", "timestamp"]], 
                    index=["label", "trans_type"], aggfunc=len)

pt.columns = ['count']
gdf = pd.DataFrame(pt.to_records())

gdf['total'] = gdf.groupby('label')['count'].transform(np.sum)
gdf['percentage'] = gdf['count'] / gdf['total']

gdf

,label,trans_type,count,total,percentage
0,fraud,chip_and_pin,2293,46396,0.049422
1,fraud,contactless,2346,46396,0.050565
2,fraud,manual,16124,46396,0.347530
3,fraud,online,23335,46396,0.502953
4,fraud,swipe,2298,46396,0.049530
5,legitimate,chip_and_pin,491303,2453604,0.200237
6,legitimate,contactless,613165,2453604,0.249904
7,legitimate,manual,122175,2453604,0.049794
8,legitimate,online,858969,2453604,0.350085
9,legitimate,swipe,367992,2453604,0.149980


In [13]:
import altair as alt

alt.Chart(gdf).mark_bar().encode(
    alt.Y('percentage:Q', axis=alt.Axis(format='.0%')), column='trans_type', x="label", color='label'
)

alt.Chart(...)

# Foreign transaction distribution

We can repeat this process for percentage of foreign vs domestic transactions for each label.

In [14]:
pt = pd.pivot_table(df[["label", "foreign", "timestamp"]], 
                    index=["label", "foreign"], aggfunc=len)

pt.columns = ['count']
gdf = pd.DataFrame(pt.to_records())

gdf['total'] = gdf.groupby('label')['count'].transform(np.sum)
gdf['pctage'] = gdf['count'] / gdf['total']

gdf

alt.Chart(gdf).mark_bar().encode(
    alt.Y('pctage:Q', axis=alt.Axis(format='.0%')), column='foreign', x="label", color='label'
)

alt.Chart(...)

# Transaction amount distribution

When we're dealing with continuous variables, rather than categorical ones, we'll need to do something other than simply counting values to make sense of their distributions (we aren't interested, for example, in knowing that there were exactly 24 transactions for \$17.35).  There are a few options for how to proceed here:

1.  we could _quantize_ values by rounding to the nearest dollar (or ten dollars, or to the nearest power of _k_ dollars),
1.  we could _discretize_ values by grouping nearby values into histogram buckets, or
1.  we could take the _cumulative distribution_ of values, which gives us a curve for the whole distribution.

Let's take a look at the latter approach.

In [15]:
qs = df[['label','amount']].groupby('label').quantile(q=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])
qs

amount
label                      
fraud      0.01    5.330000
           0.05    6.720000
           0.10    8.890000
           0.25   14.660000
           0.50   22.530001
           0.75   28.000000
           0.90   35.209999
           0.95   40.652501
           0.99   52.810001
legitimate 0.01    3.310000
           0.05    4.680000
           0.10    6.740000
           0.25   12.430000
           0.50   20.010000
           0.75   28.030001
           0.90   46.080002
           0.95   91.720001
           0.99  495.498800

What does the above data mean?  For each label, we have a set of rows, corresponding to the first, fifth, tenth, ..., ninety-fifth, and ninety-ninth percentiles.  The value at each row indicates the transaction amount at each percentile.  As an example, this means that 25% of legitimate transaction amounts are less than or equal to $12.43.

We can plot the cumulative distribution of each transaction type to compare these distributions.

In [16]:
qdf = pd.DataFrame(qs.to_records())

alt.Chart(qdf).mark_line(interpolate="monotone").encode(
    alt.Y("amount", axis=alt.Axis(title='transaction amounts (log scale)'), scale=alt.Scale(type='log')), 
    alt.X("level_0", axis=alt.Axis(title='cumulative distribution'), scale=alt.Scale(type='linear')), 
    color="label"
)

alt.Chart(...)

As we can see, relatively more of the fraudulent transactions are for low dollar amounts.

# Interarrival times

Another interesting feature to look at is the distribution of interarrival times, or the time gaps between transactions.  Our intuition is that legitimate transactions will be spread apart further (and will have some large gaps), while fraudulent transactions will cluster together.

In [17]:
fraudsamp = df[df["label"] == "fraud"].copy()
legitsamp = df[df["label"] == "legitimate"].sample(len(fraudsamp)).copy()

fraudsamp['irank'] = fraudsamp['interarrival'].rank(pct=True, method="dense")
legitsamp['irank'] = legitsamp['interarrival'].rank(pct=True, method="dense")
qdf = pd.concat([fraudsamp.groupby(['label', 'interarrival', 'irank']).size(), legitsamp.groupby(['label', 'interarrival', 'irank']).size()])
qdf = pd.DataFrame(pd.DataFrame(qdf).to_records())
qdf = qdf[qdf['interarrival'] > 0]

In [18]:
alt.Chart(qdf.sample(5000)).mark_line().encode(
    alt.Y("interarrival", axis=alt.Axis(title='interarrival time'), scale=alt.Scale(type='log')), 
    alt.X("irank", axis=alt.Axis(title='cumulative distribution'), scale=alt.Scale(type='linear')), 
    color="label"
)

alt.Chart(...)

# Activity by time of day

We've generated users to act during typical sleep-wake cycles at different times of day.  (To simplify things, the data generator assumes that a user with a user ID _k_ is in time zone _t_ if _k_ mod 24 == _t_.)  As we'll see, legitimate transactions are centered around waking hours, while fraudulent transactions are roughly equally likely to happen at any time of day.

In [19]:
df["hour"] = df["timestamp"] % 86400 // 3600
df["tz"] = df["user_id"] % 24

tzpivot = pd.pivot_table(df[["label", "hour", "tz", "timestamp"]], 
                         index=["label", "hour", "tz"], aggfunc=len)

tzdf = pd.DataFrame(tzpivot.to_records()).set_axis(["label", "tz", "hour", "count"], axis="columns")

In [20]:
alt.Chart(tzdf).mark_area().encode(
    x="hour",
    y=alt.Y("count", stack="normalize", axis=alt.Axis(title="percentage of total")), 
    color="tz",
    column="label",
    tooltip=['label', 'tz', 'hour', 'count']
)

alt.Chart(...)

Now that we've explored our data and visualised the distribution of values, let's move on to the next notebook, [02-feature-engineering.ipynb](02-feature-engineering.ipynb) where we transform the data into feature vectors. 